imports

In [1]:
import h5py
import simcat
import numpy as np
import tensorflow as tf
from numba import jit
import toytree
from copy import deepcopy
import ipyparallel as ipp

/Users/pmckenz1/miniconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import BernoulliRBM
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
import argparse
import time

In [3]:
dat=h5py.File("databases/starting_sims.hdf5")

### Make labels.

In [4]:
sources=dat['admix_sources']
targets=dat['admix_targets']

In [5]:
combo=np.hstack([sources,targets])
class_ids=np.array([str(x) for x in combo])
id_dict=dict(enumerate(np.unique(class_ids)))
inv_dict = {v: k for k, v in id_dict.iteritems()}
class_ids_int = np.zeros(class_ids.shape,dtype=np.int32)
counter = 0
for str_id in class_ids:
    class_ids_int[counter] = inv_dict[str_id]
    counter += 1
y = class_ids_int

### Get raw features (flattened)

In [18]:
def flattendb(counts):
    newshape=reduce(lambda x,y: x*y, counts[0].shape)
    X = np.zeros((counts.shape[0],newshape))
    for i in range(len(counts)):
        X[i] = counts[i].ravel()
    return(X)

In [23]:
X = flattendb(dat['counts'][0:10000])

### Scale the data.

In [8]:
def scale(X, eps = 0.001):
	# scale the data points s.t the columns of the feature space
	# (i.e the predictors) are within the range [0, 1]
	return (X - np.min(X, axis = 0)) / (np.max(X, axis = 0) + eps)

In [24]:
trainX, testX, trainY, testY = train_test_split(X, y[0:10000],
	test_size = .2, random_state = 42)

In [ ]:
# perform a grid search on the 'C' parameter of Logistic
# Regression
print "SEARCHING LOGISTIC REGRESSION"
params = {"C": [1.0, 10.0]}
start = time.time()
gs = GridSearchCV(LogisticRegression(), params, n_jobs = -1, verbose = 1)
gs.fit(trainX, trainY)

# print diagnostic information to the user and grab the
# best model
print "done in %0.3fs" % (time.time() - start)
print "best score: %0.3f" % (gs.best_score_)
print "LOGISTIC REGRESSION PARAMETERS"
bestParams = gs.best_estimator_.get_params()

# loop over the parameters and print each of them out
# so they can be manually set
for p in sorted(params.keys()):
    print "\t %s: %f" % (p, bestParams[p])

SEARCHING LOGISTIC REGRESSION
Fitting 3 folds for each of 2 candidates, totalling 6 fits


Process PoolWorker-20:
Traceback (most recent call last):
Process PoolWorker-19:
Process PoolWorker-21:
Process PoolWorker-22:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/pmckenz1/miniconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/Users/pmckenz1/miniconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/Users/pmckenz1/miniconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/Users/pmckenz1/miniconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
    self.run()
  File "/Users/pmckenz1/miniconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
    self.run()
  File "/Users/pmckenz1/miniconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
    self.run()
  File "/Users/pmckenz1/miniconda2/lib/python2.7/multiprocessing